In [ ]:
#!/usr/bin/env python
import os
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import *
from PhysicsTools.NanoAODTools.postprocessing.trigger.triggerFilter import triggerFilter

from PhysicsTools.NanoAODTools.postprocessing.modules.common.countHistogramsModule import countHistogramsProducer
from PhysicsTools.NanoAODTools.postprocessing.framework.crabhelper import runsAndLumis


In [ ]:
##
## python helper functions
##

# allows modules to be reloaded automatically without restarting kernel
%load_ext autoreload
%autoreload 2
# widens cells
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# define some parameters for running code
isMC = True
base_path = '/eos/cms/store/group/phys_exotica/bffZprime/sample_files/'
infile = '{}BFFZprimeToMuMu_M_200_dbs0p5_TuneCUETP8M1_13TeV-madgraph-pythia8_16.root'.format(base_path)
outfile = '.'
# set maxEntries = -1 to process all events
maxEntries = 10000
dataYear = '2016'
runPeriod = ''
triggers = ['HLT_Mu50', 'HLT_TkMu50', 'HLT_DoubleEle33_CaloIdL_MW', 'HLT_DoubleEle33_CaloIdL_GsfTrkIdVL']
btagWP = 0.3093 
btag_type = 'deepflavour'
selector = 'bff'
keep_and_drop = 'keep_and_drop_bff.txt'
crab = False

In [ ]:
#select right module
from PhysicsTools.NanoAODTools.postprocessing.modules.common.heepV72018PromptProducer import heepV72018PromptProducer
if isMC:
    from PhysicsTools.NanoAODTools.postprocessing.modules.btv.btagSFProducer import btagSF
    from PhysicsTools.NanoAODTools.postprocessing.modules.jme.jetmetHelperRun2 import createJMECorrector
    if dataYear=='2016':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2016 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2016 as puWeight
    if dataYear=='2017':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2017 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2017 as puWeight
    if dataYear=='2018':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2018 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2018 as puWeight
if dataYear=='2016':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2016 as muonScaleRes
if dataYear=='2017':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2017 as muonScaleRes
if dataYear=='2018':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2018 as muonScaleRes

#different preselection producerts
if selector=='inclusive':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffInclusive_preselectionModule import bffInclusivePreselProducer as preselectorProducer
if selector=='bff':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffPreselModule import bffPreselProducer as preselectorProducer
if selector=='bff_eff':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffBtagEff import bffBtagEffProducer as preselectorProducer    
if selector=='minseok':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffPreselModule_minseok import bffPreselProducer as preselectorProducer

#prepare file if glob
if not crab:
    print("using input file", infile)
    import glob
    infile = glob.glob(infile)
else:
    from PhysicsTools.NanoAODTools.postprocessing.framework.crabhelper import inputFiles
    print("get input file (crab)")
    infile = inputFiles()
print("infile", infile)
#2017 EE fix
conditions_dict = {
    '2016':{'metBranchName':'MET', 'heepBranchName': 'cutBased_HEEP'},
    '2017':{'metBranchName':'METFixEE2017', 'heepBranchName': 'cutBased_HEEP'},
    '2018':{'metBranchName':'MET', 'heepBranchName': 'cutBased_HEEPV7p0_2018Prompt'}
}


In [ ]:
#set up process for mc and data
if isMC:
    applyHEMfix=True
    jmeCorrections = createJMECorrector(
            isMC=isMC,
            dataYear=dataYear,
            runPeriod=runPeriod,
            jesUncert="Total",
            metBranchName=conditions_dict[dataYear]['metBranchName'],
            applySmearing=True,
            jetType="AK4PFchs",
            noGroom=False,
            applyHEMfix=applyHEMfix,
        )
    
    #don't use all modules for bff efficency calculation
    if 'eff' not in selector:
        modules=[
                countHistogramsProducer(),
                triggerFilter(triggers),
                btagSF(dataYear, algo=btag_type),
                jmeCorrections(),
                puWeight(),
                muonScaleRes(),
                lepSF(),
                heepV72018PromptProducer(),
                preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, applyHEMfix=applyHEMfix, **conditions_dict[dataYear])
            ]
    else:
            modules=[
                btagSF(dataYear, algo=btag_type),
                jmeCorrections(),
                preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, **conditions_dict[dataYear])
            ]
    p = PostProcessor(outfile,
            infile,
            modules=modules,
            provenance=True,
            fwkJobReport=True,
            outputbranchsel=keep_and_drop,
            maxEntries=maxEntries
        )
else:
    modules=[
            countHistogramsProducer(),
            triggerFilter(triggers),
            muonScaleRes(),
            heepV72018PromptProducer(),
            preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, **conditions_dict[dataYear])
        ]

    p = PostProcessor(outfile,
            infile,
            modules=modules,
            provenance=True,
            fwkJobReport=True,
            outputbranchsel=keep_and_drop,
            jsonInput=runsAndLumis(),
            maxEntries=maxEntries
        )

modules

In [ ]:
p.run()

In [ ]:
import uproot as upr

In [ ]:
upf = upr.open('BFFZprimeToMuMu_M_200_dbs0p5_TuneCUETP8M1_13TeV-madgraph-pythia8_16_Skim.root')['Events']

In [ ]:
df = upf.arrays(['Jet_pt_jesHEMIssueDown', 'Jet_pt_jesTotalDown', 'Jet_pt_nom', 'RelMET_nom', 'RelMET_jesHEMIssueDown' ,'SR1_nom', 'SR1_jesHEMIssueDown'], library='pd')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df[(df.SR1_nom!=df.SR1_jesHEMIssueDown) ]

In [ ]:
def get_keys(key):
    return [x for x in upf.keys() if key in x]

In [ ]:
df = upf.arrays(['HTLT_nom', 'HTLT_jesHEMIssueDown', 'SR1_nom', 'SR1_jesHEMIssueDown', 'SR1_jesTotalDown', 'Jet_pt_jesTotalDown', 'Jet_pt_jesHEMIssueDown', 'Jet_pt_nom'], library='pd')

In [ ]:
df[(df.SR1_nom!=df.SR1_jesTotalDown)]

In [ ]:
get_keys('HEM')